# UdaciSense: Optimized Object Recognition

## Notebook 4: Mobile Deployment

In this notebook, you'll prepare your optimized model for mobile deployment.
You'll explore how to convert your best optimized model to a cross-platform mobile-friendly format,
and evaluate the performance that UdaciSense mobile users can expect.

In [1]:
# Make sure that libraries are dynamically re-loaded if changed
%load_ext autoreload
%autoreload 2

In [37]:
# Import necessary libraries
import copy
import os
import json
import time
import numpy as np
import pandas as pd
import pprint
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.quantization
from typing import Dict, Any, List, Tuple, Optional, Union, Callable
import warnings

from utils.data_loader import get_household_loaders, get_input_size, print_dataloader_stats, visualize_batch
from utils.model import MobileNetV3_Household, load_model, save_model, print_model_summary
from utils.visualization import (
    plot_model_comparison, plot_multiple_models_comparison, 
    create_model_summary_dashboard
)
from utils.compression import is_quantized
from utils.evaluation import evaluate_model_metrics, compare_models

In [3]:
# Ignore PyTorch deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=torch.jit.TracerWarning)
warnings.filterwarnings("ignore", category=UserWarning)  # Optional: Ignore all user warnings

### Step 1: Set up the Environment

In [ ]:
# Check if CUDA is available
devices = ["cpu"]
if torch.cuda.is_available():
    num_devices = torch.cuda.device_count()
    devices.extend([f"cuda:{i} ({torch.cuda.get_device_name(i)})" for i in range(num_devices)])
print(f"Devices available: {devices}")

In [5]:
# Setup directories
os.makedirs("../models/mobile", exist_ok=True)
os.makedirs("../results/mobile", exist_ok=True)

In [ ]:
# Set random seed for reproducibility
def set_deterministic_mode(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    
    def seed_worker(worker_id):
        worker_seed = seed + worker_id
        np.random.seed(worker_seed)
        random.seed(worker_seed)
    
    return seed_worker

set_deterministic_mode(42)
g = torch.Generator()
g.manual_seed(42)

### Step 2: Load the dataset

In [ ]:
# Load household objects dataset
train_loader, test_loader = get_household_loaders(
    image_size="CIFAR", batch_size=128, num_workers=2,
)

# Get input_size
input_size = get_input_size("CIFAR")
print(f"Input has size: {input_size}")

# Get class names
class_names = train_loader.dataset.classes
print(f"Datasets have these classes: ")
for i in range(len(class_names)):
    print(f"  {i}: {class_names[i]}")

# Visualize some examples
for dataset_type, data_loader in [('train', train_loader), ('test', test_loader)]:
    print(f"\nInformation on {dataset_type} set")
    print_dataloader_stats(data_loader, dataset_type)
    print(f"Examples of images from the {dataset_type} set")
    visualize_batch(data_loader, num_images=10)

### Step 3. Load the optimized model and metrics

In [8]:
# TODO: Choose the experiment to load for
experiment_name = "1_post-global-pruning_dynamic-quant_torchscript"
optimized_model_path = f"../models/pipeline/{experiment_name}/model.pth"

In [ ]:
# Load the optimized model
optimized_model = load_model(optimized_model_path)
print_model_summary(optimized_model)

# Load optimized model metrics
with open(f"../results/pipeline/{experiment_name}/pipeline_metrics.json", "r") as f:
    optimized_metrics = json.load(f)

print("\nOptimized Model Metrics:")
pprint.pp(optimized_metrics)

### Step 4. Convert optimized model for mobile

In [35]:
# TODO: Implement any required torch optimizations for deployment to mobile
# Review built-in functionalities for mobile (e.g., mobile_optimizer)
def convert_model_for_mobile(
    model: nn.Module,
    input_size: Tuple[int, ...] = (1, 3, 32, 32)
) -> Union[torch.jit.ScriptModule, str]:
    """
    Convert a PyTorch model to a mobile-friendly format.
    Currently focused on TorchScript but designed to be extensible.
    
    Args:
        model: PyTorch model to convert
        input_size: Shape of input tensor (possibly useful to create a dummy input)
    Returns:
        Converted model object
    """
    
    pass

In [ ]:
# Convert the model for mobile deployment
print("\nConverting model for mobile deployment...")

# Convert the model
mobile_model = convert_model_for_mobile(
    optimized_model, 
    input_size=input_size,
    mobile_optimize=True,
)

# Save the mobile model
# EXTRA: Test saving with lite interpreter instead - do you notice differences in performance?
mobile_model_path = f"../models/mobile/optimized_model_mobile.pt"
torch.jit.save(mobile_model, mobile_model_path)
print(f"Saved mobile-compatible model to: {mobile_model_path}")

### Step 5: Verify Mobile Model Performance

Before packaging for deployment, let's verify that your optimized model meets the requirements.

#### Model output consistency

In [ ]:
# TODO: Implement checks to guarantee the same model outputs
# Once again, you can look at built-in PyTorch functionalities for inspiration
# Also consider which device to perform the operations on
def compare_model_outputs(
    model1: nn.Module,
    model2: nn.Module,
    input_tensor: torch.Tensor,
) -> bool:
    """
    Compare outputs of two models to verify consistency after conversion.
    Works with both regular PyTorch models and converted mobile models.
    
    Args:
        model1: First model
        model2: Second model
        input_tensor: Input tensor to test with
        
    Returns:
        True if outputs are consistent, False otherwise
    """
    pass
        
# Verify model output consistency
dummy_input = torch.randn(input_size)
output_consistency = compare_model_outputs(optimized_model, mobile_model, dummy_input)
print(f"Output consistency check: {'PASSED' if output_consistency else 'FAILED'}")

#### Model size

In [ ]:
# Get the model size
# Consider if you want to return both or either model size in MB vs parameter count
def get_model_size(model_path: str) -> float:
    """
    Get the size of a model, whether it is optimized or not for mobile.
    
    Args:
        model_path: Filepath to the saved model
        
    Returns:
        Float number representing model size
    """
    pass

optimized_size = get_model_size(optimized_model_path)
mobile_size = get_model_size(mobile_model_path)

print(f"Original model size: {optimized_size:.2f} MB")
print(f"Mobile model size: {mobile_size:.2f} MB")
print(f"\nSize change from optimized to mobile: {(mobile_size - optimized_size) / optimized_size * 100:.2f}%")

#### Evaluate models on test set

In [ ]:
# TODO: Evaluate original optimized vs mobile model when it comes to accuracy and other performance metrics
# Feel free to choose one or more of the built-in evaluation and visualization methods, or create new ones!

#### Benchmark Mobile Performance

In this section, you should develop a strategy for benchmarking the model on actual mobile 
hardware. Since we can't easily test on ARM mobile devices in this environment, describe:

1. What tools and frameworks you would use for mobile benchmarking?
2. What specific metrics you would collect?
3. How you would set up a fair comparison between models?
4. What mobile-specific factors you would control for in your tests?

Write your benchmarking approach in the final report directly.

--------

**TODO: Collect results on mobile conversion and considerations for model deployment**

After converting your optimized model for mobile deployment, analyze how the model will perform in real-world mobile environments.

Consider these guiding questions:
- How did the mobile conversion affect the model's performance characteristics?
- What mobile-specific considerations impact this model's deployment?
- How would you rigorously benchmark performance across different devices?
- What challenges might arise in production deployment scenarios?
- What future improvements would you prioritize for model deployment?

Provide an analysis that demonstrates your understanding of mobile deployment considerations for the UdaciSense computer vision model.

## Optimized Model Mobile Deployment Analysis for UdaciSense Computer Vision Model

*Replace this with your analysis*

> 🚀 **Next Step:** 
> Collect all your results and insights in `report.md`! 